In [183]:
from pathlib import Path

DATASET = Path("../data/masked.json")

MODEL = "ufal/robeczech-base"

In [184]:
import pandas as pd
from datasets import Dataset

df = pd.read_json(DATASET)
df.head()
dataset = Dataset.from_pandas(df)

In [185]:
from transformers import pipeline

corrector = pipeline("fill-mask", model=MODEL, tokenizer=MODEL, device=0, top_k=3)

### Example usage

In [186]:
masked_sentences = dataset["masked"]

for predictions in corrector(masked_sentences[0]):
    print(predictions[:2])

[{'score': 0.961595356464386, 'token': 1469, 'token_str': ' třídy', 'sequence': 'Mercedes-Benz W123 je automobil vyšší střední třídy vyráběný mezi letny 1976 až 1985.'}, {'score': 0.03217912092804909, 'token': 2664, 'token_str': ' kategorie', 'sequence': 'Mercedes-Benz W123 je automobil vyšší střední kategorie vyráběný mezi letny 1976 až 1985.'}]
[{'score': 0.6241330504417419, 'token': 596, 'token_str': ' lety', 'sequence': 'Mercedes-Benz W123 je automobil vyšší střední třdy vyráběný mezi lety 1976 až 1985.'}, {'score': 0.21872220933437347, 'token': 546, 'token_str': ' roky', 'sequence': 'Mercedes-Benz W123 je automobil vyšší střední třdy vyráběný mezi roky 1976 až 1985.'}]


In [187]:
SHARDS = 2

In [188]:
from tqdm.auto import tqdm

def explode_masked_and_fix(df: pd.DataFrame) -> pd.DataFrame:
    df = df.explode("masked")
    df["fix"] = None
    df.reset_index(drop=True, inplace=True)
    return df

def _merge_masked(row):
    sentence = row["sentence"].split(' ')
    for masked in row["masked"]:
        sentence[masked.split(" ").index("[MASK]")] = "[MASK]"

    return row["sentence"], " ".join(sentence), row["fix"]

def implode_and_merge_masked(df: pd.DataFrame) -> pd.DataFrame:
    df = df.groupby(["sentence"]).agg(
        {
            "masked": list,
            "fix": list
        }
    ).reset_index()

    df["sentence"], df["masked"], df["fix"] = zip(*df.apply(_merge_masked, axis=1))
    return df

def create_dataset() -> pd.DataFrame:
    final_df = pd.DataFrame()

    for i in tqdm(range(SHARDS)):
        _dataset = dataset.shard(num_shards=SHARDS, index=i)

        df = _dataset.to_pandas()
        df = explode_masked_and_fix(df)

        predictions = corrector(df["masked"].to_list(), batch_size=32)

        for j, prediction in enumerate(predictions):
            df.loc[j, "fix"] = prediction[0]["token_str"].strip()

        df = implode_and_merge_masked(df)

        final_df = pd.concat([final_df, df], ignore_index=True)

    return final_df
    
    

### Experiment 1
We will replace `[MASK]` with the suggestion that has biggest score.

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  4.79it/s]


,sentence,masked,fix
0,"Bezpečné rozlišení obou druhů je často velmi složité , proto se sběr openěk doporučuje jen zkušenějším mykologům .","Bezpečné rozlišení obou druhů [MASK] často velmi složité , proto se sběr [MASK] doporučuje jen zkušenějším [MASK] .","[je, vzorků, uživatelům]"
1,Byly s ní prováděny experimenty za účelem urychlení rozkladu pařezů .,Byly s ní prováděny [MASK] za [MASK] urychlení rozkladu [MASK] .,"[práce, účelem, tkání]"
2,Doporučuje se buk a habr .,Doporučuje se buk a habr [MASK],[.]
3,"Dužnina v klobouku je tenká hygrofánní , měkká a bělavá , v třeni pevná a rezavě hnědá , vůni má příjemnou , nasládle kořenitou , chuť lahodnou .","Dužnina v [MASK] je tenká hygrofánní , měkká a bělavá , v třeni pevná a rezavě hnědá , vůni má [MASK] , nasládle kořenitou , [MASK] lahodnou .","[plodu, příjemnou, sladkou]"
4,Je též oblíbenou ingrediencí gulášů a masitých pokrmů .,Je též [MASK] ingrediencí gulášů a [MASK] pokrmů .,"[hlavní, jiných]"
